<a href="https://colab.research.google.com/github/asu-trans-ai-lab/CGChoice/blob/master/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cvxpylayers

     |████████████████████████████████| 2.2MB 8.9MB/s 
     |████████████████████████████████| 1.3MB 35.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.11-cp37-cp37m-linux_x86_64.whl size=2725449 sha256=97e5587f04a8d1785d4630855fd4de4b427b07e335506398f783c0b0fbede97d
  Stored in directory: /root/.cache/pip/wheels/df/d4/eb/c74e09a39bc4ac09b3a8d2a48c1c6009f391056d8c87bf68aa
Successfully built cvxpy
  Created wheel for cvxpylayers: filename=cvxpylayers-0.1.4-cp37-none-any.whl size=18943 sha256=8935e22552dba7d29430bbfa308fa535e787e39309b482e1d7cfe7e67dab44ba
  Stored in directory: /root/.cache/pip/wheels/a9/a4/9a/4c426a86d6b507b8279f0bb54b9bbbb6ac2eefdd1dd1b0fdd6
  Created wheel for diffcp: filename=diffcp-1.0.16-cp37-cp37m-linux_x86_64.whl size=3982945 sha256=d6a5c198c7e0cf22c33eac9544ca0ab736036336caab825eaa6a34b6afbc4063
  Stored in directory: /root/.cache/p

In [2]:
import cvxpy as cp
import tensorflow as tf
from cvxpylayers.tensorflow import CvxpyLayer

In [3]:
n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])
A_tf = tf.Variable(tf.random.normal((m, n)))
b_tf = tf.Variable(tf.random.normal((m,)))

with tf.GradientTape() as tape:
  # solve the problem, setting the values of A, b to A_tf, b_tf
  solution, = cvxpylayer(A_tf, b_tf)
  summed_solution = tf.math.reduce_sum(solution)
# compute the gradient of the summed solution with respect to A, b
gradA, gradb = tape.gradient(summed_solution, [A_tf, b_tf])

In [5]:
summed_solution

<tf.Tensor: shape=(), dtype=float64, numpy=0.45507420192913267>

In [4]:
gradA

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[-2.94053145e-01,  2.89748914e-11],
       [ 4.28637520e-12, -3.30627563e-12],
       [ 6.71237595e-12, -3.30627563e-12]])>